In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.callbacks import get_openai_callback


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
groq_api_key = os.getenv("GROQ_API_KEY")

MODEL_NAME = "mixtral-8x7b-32768"

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key
)

In [10]:
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a cpecified city
    
    Args:
        city (str): The name of the city where you want to check the weather
        
    Returns:
        str: A description of the current weather in a specified city
    """
    return "Sunny, 33°C"


tools = [multiply, add, fake_weather_api]

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant skilled in math operations."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [12]:
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
result = agent_executor.invoke({"input": "What's 3 plus 5 multiplied by 2?"})
print(result)



> Entering new AgentExecutor chain...



Invoking: `add` with `{'x': 3, 'y': 5}`


8.0
Invoking: `multiply` with `{'x': 8.0, 'y': 2}`


16.0The final answer to the question "What's 3 plus 5 multiplied by 2?" is 16.0.

> Finished chain.
{'input': "What's 3 plus 5 multiplied by 2?", 'output': 'The final answer to the question "What\'s 3 plus 5 multiplied by 2?" is 16.0.'}


In [14]:
result = agent_executor.invoke({"input": "What's the current weather in Tel-Aviv?"})
print(result)



> Entering new AgentExecutor chain...



Invoking: `fake_weather_api` with `{'city': 'Tel-Aviv'}`


Sunny, 33°CThe current weather in Tel-Aviv is Sunny, 33°C.

> Finished chain.
{'input': "What's the current weather in Tel-Aviv?", 'output': 'The current weather in Tel-Aviv is Sunny, 33°C.'}


In [15]:
result

{'input': "What's the current weather in Tel-Aviv?",
 'output': 'The current weather in Tel-Aviv is Sunny, 33°C.'}

In [16]:
from langchain_core.callbacks.base import BaseCallbackHandler

class PromptPrintingCallbackHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print("\nFull Prompt:")
        for prompt in prompts:
            print(prompt)

# Use the custom callback handler
callback_handler = PromptPrintingCallbackHandler()

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key,
    callbacks=[callback_handler]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


result = agent_executor.invoke(
    {"input": "What's 3 plus 5 multiplied by 2?"}
)

print(result)



> Entering new AgentExecutor chain...

Full Prompt:
System: You're a helpful assistant skilled in math operations.
Human: What's 3 plus 5 multiplied by 2?

Invoking: `add` with `{'x': 3, 'y': 5}`


8.0
Full Prompt:
System: You're a helpful assistant skilled in math operations.
Human: What's 3 plus 5 multiplied by 2?
AI: 
Tool: 8.0

Invoking: `multiply` with `{'x': 8.0, 'y': 2}`


16.0
Full Prompt:
System: You're a helpful assistant skilled in math operations.
Human: What's 3 plus 5 multiplied by 2?
AI: 
Tool: 8.0
AI: 
Tool: 16.0
The final answer to the question "What's 3 plus 5 multiplied by 2?" is 16.0.

> Finished chain.
{'input': "What's 3 plus 5 multiplied by 2?", 'output': 'The final answer to the question "What\'s 3 plus 5 multiplied by 2?" is 16.0.'}
